Using CLIP's image encoder for feature extraction, training all the images and the 512 dimension embedding is used for training the ML models

In [1]:
import os
import clip
import torch
import numpy as np
from tqdm import tqdm
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from skimage import io, transform

In [2]:
#Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

In [3]:
root_pth = './birds/CUB_200_2011/images/'
imgID_pth_df = pd.read_csv('./birds/CUB_200_2011/images.txt', sep=' ', header=None, names=['img_id', 'sub_pth'])
train_split_df = pd.read_csv('./birds/CUB_200_2011/train_test_split.txt', sep=' ', names=['img_id', 'is_training'])
class_names = pd.read_csv('./birds/CUB_200_2011/classes.txt', sep=' ', header=None, names=['class_id', 'class_name'])
class_names.class_name = class_names.class_name.map(lambda x: x.split('.')[1].lower())

cub_dataset_df = imgID_pth_df.merge(train_split_df, on='img_id', how='inner')


target=[]
for pth in cub_dataset_df['sub_pth']:
    target.append(int(pth.split('.')[0]))
df = pd.DataFrame(target, columns=['class'])
train_df=cub_dataset_df[cub_dataset_df.is_training==1]
final_df = train_df.join(df)

whole_df=pd.DataFrame(columns=['img_id', 'sub_pth', 'is_training', 'class'])
for i in range(1,201):
    whole=final_df[final_df['class']==i]
    whole_df=whole_df.append(whole, ignore_index=True)

whole_df


/tmp/ipykernel_4076287/3077369022.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  whole_df=whole_df.append(whole, ignore_index=True)


,img_id,sub_pth,is_training,class
0,2,001.Black_footed_Albatross/Black_Footed_Albatr...,1,1
1,4,001.Black_footed_Albatross/Black_Footed_Albatr...,1,1
2,5,001.Black_footed_Albatross/Black_Footed_Albatr...,1,1
3,7,001.Black_footed_Albatross/Black_Footed_Albatr...,1,1
4,8,001.Black_footed_Albatross/Black_Footed_Albatr...,1,1
...,...,...,...,...
5989,11778,200.Common_Yellowthroat/Common_Yellowthroat_00...,1,200
5990,11781,200.Common_Yellowthroat/Common_Yellowthroat_00...,1,200
5991,11782,200.Common_Yellowthroat/Common_Yellowthroat_00...,1,200
5992,11784,200.Common_Yellowthroat/Common_Yellowthroat_00...,1,200


In [4]:

class CUBDataset(Dataset):
    """CUB dataset."""

    def __init__(self, dataframe, root_dir, transform=None):
        """
        Args:
            dataframe (pd.DataFrame): Dataframe with paths and train/test split information.
            root_dir (string): Root directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.cub_img_df = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.cub_img_df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        sub_pth = self.cub_img_df.iloc[idx, 1]
        img_name = os.path.join(self.root_dir,
                                sub_pth)
        image = io.imread(img_name)
        target = int(sub_pth.split('.')[0])#.split('.')[1].lower()
        if self.transform:
            image = self.transform(image)
        return image, target

# loading CUB-200-2011 train&test dataset
train_set = CUBDataset(whole_df, root_pth, transform=transforms.Compose([transforms.ToPILImage(), preprocess]))
test_set = CUBDataset(cub_dataset_df[cub_dataset_df.is_training==0], root_pth, transform=transforms.Compose([transforms.ToPILImage(), preprocess]))

In [5]:
def get_features(dataset):
    all_features = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(DataLoader(dataset, batch_size=16,shuffle=True)):
            features = model.encode_image(images.to(device))
            #features /= features.norm(dim=-1, keepdim=True)
            all_features.append(features)
            all_labels.append(labels)
    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

In [6]:
train_features, train_labels = get_features(train_set)
test_features, test_labels = get_features(test_set)

100%|██████████| 363/363 [00:59<00:00,  6.08it/s]


In [7]:
print(train_features.shape, test_features.shape)

(5994, 512) (5794, 512)


In [8]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(train_features, train_labels)
predictions = classifier.predict(test_features)

accuracy = np.mean((test_labels == predictions).astype(np.float)) * 100.
print(f"Accuracy Logistic = {accuracy:.3f}")

from sklearn import neighbors
clf=neighbors.KNeighborsClassifier()
clf.fit(train_features, train_labels)
predictions = clf.predict(test_features)


accuracy = np.mean((test_labels == predictions).astype(np.float)) * 100.
print(f"Accuracy KNN= {accuracy:.3f}")

from sklearn.svm import SVC
scl=SVC(kernel='linear')
scl.fit(train_features, train_labels)
predictions = scl.predict(test_features)
accuracy = np.mean((test_labels == predictions).astype(np.float)) * 100.
print(f"Accuracy SVM-Linear= {accuracy:.3f}")

from sklearn.svm import SVC
sc=SVC(kernel='rbf')
sc.fit(train_features, train_labels)
predictions = sc.predict(test_features)
accuracy = np.mean((test_labels == predictions).astype(np.float)) * 100.
print(f"Accuracy SVM-RBF= {accuracy:.3f}")

/home/santosh.sanjeev/.conda/envs/AIproject/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/tmp/ipykernel_4076287/2153442391.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  accuracy = np.mean((test_labels == predic

Accuracy Logistic = 74.284


/tmp/ipykernel_4076287/2153442391.py:15: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  accuracy = np.mean((test_labels == predictions).astype(np.float)) * 100.


Accuracy KNN= 57.784


/tmp/ipykernel_4076287/2153442391.py:22: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  accuracy = np.mean((test_labels == predictions).astype(np.float)) * 100.


Accuracy SVM-Linear= 72.938
Accuracy SVM-RBF= 71.712


/tmp/ipykernel_4076287/2153442391.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  accuracy = np.mean((test_labels == predictions).astype(np.float)) * 100.
